In [12]:
import os
import cv2
import numpy as np
from skimage import feature
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [13]:
def find_faces_by_CascadeClassifier(image_path, output_path):
    """
    Обнаружение лиц с помощью каскадного классификатора Haar
    """

    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка загрузки изображения: {image_path}")
        return
    
    image_with_boxes = image.copy()
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    
    for (x, y, w, h) in faces:
        cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.imwrite(output_path, image_with_boxes)
    
    return len(faces)

In [14]:
def find_people_by_hog(image_path, output_path):
    """
    Обнаружение людей с помощью HOG (Histogram of Oriented Gradients) + SVM
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка загрузки изображения: {image_path}")
        return
    
    image_with_boxes = image.copy()
    
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
    boxes, weights = hog.detectMultiScale(
        image,
        winStride=(8, 8),
        padding=(32, 32),
        scale=1.05,
        hitThreshold=0.5
    )
    
    people_count = 0
    for i, (x, y, w, h) in enumerate(boxes):
        if weights[i] > 0.5:  # Фильтрация по уверенности
            cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), (255, 0, 0), 2)
            people_count += 1
    
    cv2.imwrite(output_path, image_with_boxes)
    
    return people_count

In [15]:
def find_people_by_yolo(image_path, output_path):
    """
    Обнаружение людей с помощью YOLO
    """
    # Загрузка предварительно обученной модели YOLO
    try:
        model = YOLO('yolov8n.pt')  # Можно использовать 'yolov8s.pt', 'yolov8m.pt' и т.д.
    except Exception as e:
        print(f"Ошибка загрузки модели YOLO: {e}")
        return 0
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка загрузки изображения: {image_path}")
        return 0
    
    results = model(image)
    
    image_with_boxes = image.copy()
    
    people_count = 0
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Получение координат и класса
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())
            
            # Класс 'person' в COCO dataset имеет индекс 0
            if cls == 0 and conf > 0.5:  # Фильтрация по уверенности
                people_count += 1
                cv2.rectangle(image_with_boxes, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
    
    cv2.imwrite(output_path, image_with_boxes)
    
    return people_count

In [16]:
def process_directory(input_dir, output_dirs, methods):
    """
    Обработка всех изображений в каталоге с помощью указанных методов
    """
    # Создание выходных каталогов если они не существуют
    for output_dir in output_dirs.values():
        os.makedirs(output_dir, exist_ok=True)
    
    # Получение списка изображений
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    image_files = [f for f in os.listdir(input_dir) 
                  if os.path.splitext(f)[1].lower() in image_extensions]
    
    print(f"Найдено {len(image_files)} изображений для обработки")
    
    # Статистика
    stats = {method: 0 for method in methods.keys()}
    
    # Обработка каждого изображения
    for image_file in image_files:
        input_path = os.path.join(input_dir, image_file)
        filename = os.path.splitext(image_file)[0]
        
        for method_name, method_func in methods.items():
            output_path = os.path.join(output_dirs[method_name], f"{filename}_detected.jpg")
            try:
                count = method_func(input_path, output_path)
                stats[method_name] += count
            except Exception as e:
                print(f"Ошибка при обработке {image_file} методом {method_name}: {e}")
    
    # Вывод статистики
    print("\n=== СТАТИСТИКА ОБНАРУЖЕНИЯ ===")
    for method_name, total_count in stats.items():
        print(f"{method_name}: обнаружено {total_count} объектов")

In [17]:
input_directory = "data/src"  # Замените на путь к вашему каталогу с изображениями
output_directories = {
        "cascade": "data/cascade",
        "hog": "data/hog", 
        "yolo": "data/yolo"
    }

detection_methods = {
        "cascade": find_faces_by_CascadeClassifier,
        "hog": find_people_by_hog,
        "yolo": find_people_by_yolo
    }

process_directory(input_directory, output_directories, detection_methods)

Найдено 14 изображений для обработки

0: 480x640 5 persons, 1 cup, 1 bowl, 1 chair, 1 oven, 1 refrigerator, 79.7ms
Speed: 3.1ms preprocess, 79.7ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 persons, 72.7ms
Speed: 2.6ms preprocess, 72.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 6 cups, 1 fork, 5 bowls, 1 sandwich, 1 dining table, 1 refrigerator, 71.7ms
Speed: 3.1ms preprocess, 71.7ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 640x480 5 persons, 68.2ms
Speed: 2.6ms preprocess, 68.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 480)

0: 480x640 3 persons, 2 cars, 73.6ms
Speed: 3.2ms preprocess, 73.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 20 persons, 74.1ms
Speed: 2.0ms preprocess, 74.1ms inference, 4.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 76.6ms
Speed: 3.2ms preprocess, 76.6ms inference, 1.